In [0]:
# STARTER CODE - DO NOT EDIT THIS CELL

from pyspark.sql.functions import*
from pyspark.sql.types import *
from pyspark.sql import Window
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


In [0]:
# STARTER CODE - DO NOT EDIT THIS CELL

#from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

customSchema = StructType([
    StructField("lpep_pickup_datetime", StringType(), True),
    StructField("lpep_dropoff_datetime", StringType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", FloatType(), True),
    StructField("fare_amount", FloatType(), True),
    StructField("payment_type", IntegerType(), True)
])

In [0]:
# STARTER CODE - YOU CAN LOAD ANY FILE WITH A SIMILAR SYNTAX.
# Correct file path for Databricks File System (update if you are using a different volume or file name)
file_path = "/Volumes/workspace/default/q2vol/nyc-tripdata.csv"

# Read the CSV file using Spark DataFrame
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

display(df.limit(5))


lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,payment_type
12/21/2018 15:17,12/21/2018 15:18,264,264,5,0.0,3.0,2
01/01/2019 0:10,01/01/2019 0:16,97,49,2,0.86,6.0,2
01/01/2019 0:27,01/01/2019 0:31,49,189,2,0.66,4.5,1
01/01/2019 0:46,01/01/2019 1:04,189,17,2,2.68,13.5,1
01/01/2019 0:19,01/01/2019 0:39,82,258,1,4.53,18.0,2


In [0]:
# LOAD THE "taxi_zone_lookup.csv" FILE SIMILARLY AS ABOVE. CAST ANY COLUMN TO APPROPRIATE DATA TYPE IF NECESSARY.

#ENTER THE CODE BELOW
# Correct file path for the taxi zone lookup dataset
zone_file_path = "/Volumes/workspace/default/q2vol/taxi_zone_lookup.csv"

# Read the CSV file using Spark DataFrame
taxi_zone_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(zone_file_path)

# (Optional) Cast specific columns if necessary, e.g., LocationID to Integer
from pyspark.sql.functions import col

taxi_zone_df = taxi_zone_df.withColumn("LocationID", col("LocationID").cast("int"))

display(taxi_zone_df.limit(5))



LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


In [0]:
#// STARTER CODE 
# // Some commands that you can use to see your dataframes and results of the operations. You can comment the df.show(5) and uncomment display(df) to see the data differently. You will find these two functions useful in reporting your results.
df.show(5)
#display(df.limit(5))

+--------------------+---------------------+------------+------------+---------------+-------------+-----------+------------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|payment_type|
+--------------------+---------------------+------------+------------+---------------+-------------+-----------+------------+
|    12/21/2018 15:17|     12/21/2018 15:18|         264|         264|              5|          0.0|        3.0|           2|
|     01/01/2019 0:10|      01/01/2019 0:16|          97|          49|              2|         0.86|        6.0|           2|
|     01/01/2019 0:27|      01/01/2019 0:31|          49|         189|              2|         0.66|        4.5|           1|
|     01/01/2019 0:46|      01/01/2019 1:04|         189|          17|              2|         2.68|       13.5|           1|
|     01/01/2019 0:19|      01/01/2019 0:39|          82|         258|              1|         4.53|       18.0|      

In [0]:
# // STARTER CODE - DO NOT EDIT THIS CELL
# Filter the data to only keep the rows where "PULocationID" and the "DOLocationID" are different and the "trip_distance" is strictly greater than 2.0 (>2.0).

# VERY VERY IMPORTANT: ALL THE SUBSEQUENT OPERATIONS MUST BE PERFORMED ON THIS FILTERED DATA

df_filter = df.filter((df.PULocationID != df.DOLocationID) & (df.trip_distance > 2.0))
df_filter.show(5)

+--------------------+---------------------+------------+------------+---------------+-------------+-----------+------------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|payment_type|
+--------------------+---------------------+------------+------------+---------------+-------------+-----------+------------+
|     01/01/2019 0:46|      01/01/2019 1:04|         189|          17|              2|         2.68|       13.5|           1|
|     01/01/2019 0:19|      01/01/2019 0:39|          82|         258|              1|         4.53|       18.0|           2|
|     01/01/2019 0:47|      01/01/2019 1:00|         255|          33|              1|         3.77|       13.5|           1|
|     01/01/2019 0:12|      01/01/2019 0:30|          76|         225|              1|          4.1|       16.0|           1|
|     01/01/2019 0:16|      01/01/2019 0:39|          25|          89|              1|         7.75|       25.5|      

In [0]:
# PART 1a: The top-5 most popular drop locations - "DOLocationID", sorted in descending order - if there is a tie, then one with lower "DOLocationID" gets listed first

# Output Schema: DOLocationID int, number_of_dropoffs int 

# Hint: Checkout the groupBy(), orderBy() and count() functions.

# ENTER THE CODE BELOW
from pyspark.sql.functions import col, count

# Group by DOLocationID and count number of dropoffs
top_dropoffs = (
    df_filter.groupBy("DOLocationID")
    .agg(count("*").alias("number_of_dropoffs"))
    .orderBy(col("number_of_dropoffs").desc(), col("DOLocationID").asc())
    .limit(5)
)

display(top_dropoffs)



DOLocationID,number_of_dropoffs
61,5937
138,5146
239,4133
244,4006
42,3859


In [0]:
# PART 1b: The top-5 most popular pickup locations - "PULocationID", sorted in descending order - if there is a tie, then one with lower "PULocationID" gets listed first 

# Output Schema: PULocationID int, number_of_pickups int

# ENTER THE CODE BELOW
from pyspark.sql.functions import col, count

# Group by PULocationID and count number of pickups
top_pickups = (
    df_filter.groupBy("PULocationID")
    .agg(count("*").alias("number_of_pickups"))
    .orderBy(col("number_of_pickups").desc(), col("PULocationID").asc())
    .limit(5)
)

display(top_pickups)



PULocationID,number_of_pickups
74,17360
75,13299
244,9958
41,9645
82,9306


In [0]:
# PART 2: List the top-3 locations with the maximum overall activity, i.e. sum of all pickups and all dropoffs at that LocationID. In case of a tie, the lower LocationID gets listed first.

# Output Schema: LocationID int, number_activities int

# Hint: In order to get the result, you may need to perform a join operation between the two dataframes that you created in earlier parts (to come up with the sum of the number of pickups and dropoffs on each location). 

# ENTER THE CODE BELOW
from pyspark.sql.functions import col, count, coalesce

# Count pickups by PULocationID
pickup_counts = (
    df_filter.groupBy("PULocationID")
    .agg(count("*").alias("number_of_pickups"))
)

# Count dropoffs by DOLocationID
dropoff_counts = (
    df_filter.groupBy("DOLocationID")
    .agg(count("*").alias("number_of_dropoffs"))
)

# Join on matching LocationIDs (outer join ensures all locations are included)
activity_df = (
    pickup_counts.join(
        dropoff_counts,
        pickup_counts.PULocationID == dropoff_counts.DOLocationID,
        "outer"
    )
    .select(
        coalesce(col("PULocationID"), col("DOLocationID")).alias("LocationID"),
        (coalesce(col("number_of_pickups"), col("number_of_dropoffs")) +
         coalesce(col("number_of_dropoffs"), col("number_of_pickups"))).alias("number_activities")
    )
)

# Sort descending by activity count, tie-break by lower LocationID, and limit to top 3
top_activity = activity_df.orderBy(col("number_activities").desc(), col("LocationID").asc()).limit(3)

display(top_activity)




LocationID,number_activities
74,20292
75,16326
244,13964


In [0]:
# PART 3: List all the boroughs (including "Unknown" and "EWR") in the order of having the highest to lowest number of activities (i.e. sum of all pickups and all dropoffs at that LocationID), along with the total number of activity counts for each borough in NYC during that entire period of time.

# Output Schema: Borough string, total_number_activities int

# Hint: You can use the dataframe obtained from the previous part, and will need to do the join with the 'taxi_zone_lookup' dataframe. Also, checkout the "agg" function applied to a grouped dataframe.

# ENTER THE CODE BELOW
from pyspark.sql.functions import col, sum as Fsum

# Build per-LocationID counts for pickups and dropoffs, then union and aggregate
pickup_counts = (
    df_filter.groupBy("PULocationID").count()
    .select(col("PULocationID").alias("LocationID"), col("count").alias("cnt"))
)

dropoff_counts = (
    df_filter.groupBy("DOLocationID").count()
    .select(col("DOLocationID").alias("LocationID"), col("count").alias("cnt"))
)

activity_df = (
    pickup_counts.unionByName(dropoff_counts)
    .groupBy("LocationID")
    .agg(Fsum("cnt").alias("number_activities"))
)

# Join with taxi zone lookup to get Borough, aggregate activities per Borough
borough_activity = (
    taxi_zone_df.select("LocationID", "Borough")
    .join(activity_df, "LocationID", "left")
    .groupBy("Borough")
    .agg(Fsum("number_activities").alias("total_number_activities"))
    .orderBy(col("total_number_activities").desc(), col("Borough").asc())
)

display(borough_activity)



In [0]:
# PART 4: List the top 2 days of week with the largest number of daily average pickups, along with the average number of pickups on each of the 2 days in descending order (no rounding off required). Here, the average pickup is calculated by taking an average of the number of pick-ups on different dates falling on the same day of the week. For example, 02/01/2021, 02/08/2021 and 02/15/2021 are all Mondays, so the average pick-ups for these is the sum of the pickups on each date divided by 3.

# Note: The day of week is a string of the day’s full spelling, e.g., "Monday" instead of the		number 1 or "Mon". Also, the pickup_datetime is in the format: yyyy-mm-dd.

# Output Schema: day_of_week string, avg_count float

# Hint: You may need to group by the "date" (without time stamp - time in the day) first. Checkout "date_format" and "to_date" functions.

# ENTER THE CODE BELOW
# Use df_filter as mandated in the instructions
# Parse pickup timestamp, derive date, count daily pickups, then average by weekday
pickup_with_date = (
    df_filter
    .withColumn("pickup_ts", to_timestamp(col("lpep_pickup_datetime"), "MM/dd/yyyy H:mm"))
    .withColumn("pickup_date", to_date(col("pickup_ts")))
)

daily_counts = (
    pickup_with_date
    .groupBy("pickup_date")
    .agg(count("*").alias("daily_pickups"))
)

result_part4 = (
    daily_counts
    .withColumn("day_of_week", date_format(col("pickup_date"), "EEEE"))
    .groupBy("day_of_week")
    .agg(avg(col("daily_pickups")).alias("avg_count"))
    .orderBy(col("avg_count").desc())
    .limit(2)
)

display(result_part4)



In [0]:
# PART 5: For each particular hour of a day (0 to 23, 0 being midnight) - in their order from 0 to 23, find the zone in Brooklyn borough with the LARGEST number of pickups. 

# Note: All dates for each hour should be included.

# Output Schema: hour_of_day int, zone string, max_count int

# Hint: You may need to use "Window" over hour of day, along with "group by" to find the MAXIMUM count of pickups

# ENTER THE CODE BELOW
# Join pickups with taxi zone lookup, filter to Brooklyn, compute hour, and pick max per hour
pickups_brooklyn = (
    df_filter
    .join(taxi_zone_df.select("LocationID", "Borough", "Zone"), df_filter.PULocationID == taxi_zone_df.LocationID, "inner")
    .filter(col("Borough") == "Brooklyn")
    .withColumn("pickup_ts", to_timestamp(col("lpep_pickup_datetime"), "MM/dd/yyyy H:mm"))
    .withColumn("hour_of_day", hour(col("pickup_ts")))
)

counts = (
    pickups_brooklyn
    .groupBy("hour_of_day", "Zone")
    .agg(count("*").alias("cnt"))
)

w = Window.partitionBy("hour_of_day").orderBy(col("cnt").desc(), col("Zone").asc())

result_part5 = (
    counts
    .withColumn("rn", row_number().over(w))
    .filter(col("rn") == 1)
    .select(col("hour_of_day").cast("int").alias("hour_of_day"), col("Zone").alias("zone"), col("cnt").alias("max_count"))
    .orderBy(col("hour_of_day").asc())
)

display(result_part5)


In [0]:
# PART 6 - Find which 3 different days in the month of January, in Manhattan, saw the largest positive percentage increase in pick-ups compared to the previous day, in the order from largest percentage increase to smallest percentage increase 

# Note: All years need to be aggregated to calculate the pickups for a specific day of January. The change from Dec 31 to Jan 1 can be excluded.

# Output Schema: day int, percent_change float

# Hint: You might need to use lag function, over a window ordered by day of month.

# ENTER THE CODE BELOW
# Restrict to Manhattan pickups, parse pickup timestamp, keep January only
manhattan_pickups = (
    df_filter
    .join(taxi_zone_df.select("LocationID", "Borough"), df_filter.PULocationID == taxi_zone_df.LocationID, "inner")
    .filter(col("Borough") == "Manhattan")
    .withColumn("pickup_ts", to_timestamp(col("lpep_pickup_datetime"), "MM/dd/yyyy H:mm"))
    .filter(month(col("pickup_ts")) == 1)
    .withColumn("day", dayofmonth(col("pickup_ts")))
)

# Total pickups per day of January across all years
daily = (
    manhattan_pickups
    .groupBy("day")
    .agg(count("*").alias("pickups"))
)

# Compute percent change vs previous day number (exclude Jan 1 automatically via NULL prev)
w = Window.orderBy(col("day").asc())

result_part6 = (
    daily
    .withColumn("prev_pickups", lag("pickups").over(w))
    .withColumn(
        "percent_change",
        when(col("prev_pickups") > 0, (col("pickups") - col("prev_pickups")) * 100.0 / col("prev_pickups"))
    )
    .filter(col("percent_change").isNotNull() & (col("percent_change") > 0))
    .orderBy(col("percent_change").desc(), col("day").asc())
    .select(col("day").cast("int").alias("day"), col("percent_change"))
    .limit(3)
)

display(result_part6)

